Requirements:
- [Get Started](../01-Day_1/02-Get_started.ipynb)
- [Data management](../01-Day_1/04-Data_management.ipynb)

# Deep copy with manual attachment

We already saw how to deal with complex data structures using deep copy with implicit attach behavior in [this notebook](../03-Day_3/03-Deep_copy.ipynb). 

It is also possible to proceed to a bottom-up deep copy, in which you can first copy sub-objects on the accelerator and then attach them to existing children.
With this procedure you will have to attach explicitly the pointers to the children. This can be easily apprehend with the subsequent code.

```fortran
type vect2D
 real, allocatable, target :: x(:), y(:)
 real, pointer             :: coordinate(:)
end type

type(vect2D) :: v0

!$acc enter data copyin(v0%x, v0%y, v0%coordinate)
!$acc enter data copyin(v0)
```

Here the first copyin will pass the arrays on the device memory and the second will provide the complex datatype. But the pointers of the complex datatype (such as v0.x) will still reference the host structure.
We should thus provide to the compiler the information of the datatype as it should be on the device. This is done by adding an `attach` directive.

```fortran
type vect2D
 real, allocatable, target :: x(:), y(:)
 real, pointer :: coordinate(:)
end type

type(vect2D) :: v0

!$acc enter data copyin(v0%x, v0%y, v0%coordinate)
!$acc enter data copyin(v0) attach(v0%x, v0%y, v0%coordinate)
```

Here, the pointer and its target are present on the device. The directive will inform the compiler to replace the host pointer with the corresponding device pointer in device memory. 

The `detach` clause can be use to free the structure memory but is not mandatory.

```fortran
!$acc exit data detach(v0%x, v0%y, p1%coordinate)    ! not required
!$acc exit data copyout(v0%x, v0%y, p1%coordinate)
!$acc exit data copyout(v0)
```

## Exercise

In [ ]:
%%idrrun -a
!!  examples/Fortran/Deep_copy_attach_detach_exercise.f90
module utils_lokta
    use ISO_FORTRAN_ENV, only : REAL64, INT32
    implicit none

    type :: population
        real(kind=REAL64),dimension(:),allocatable :: state
    end type          

    contains
    subroutine derivee(x, dx, pop)
        type   (population ), dimension(2), intent(in)  :: pop 
        real   (kind=REAL64), dimension(2), intent(in)  :: x
        real   (kind=REAL64), dimension(2), intent(out) :: dx
        ! Add openacc directives
        dx(1) =  pop(1)%state(2)*x(1) - pop(1)%state(3)*x(1)*x(2)
        dx(2) = -pop(2)%state(3)*x(2) + pop(2)%state(2)*x(1)*x(2)
    end subroutine derivee

    subroutine rk4(pop, dt)
        type   (population ), dimension(2), intent(inout) :: pop    
        real   (kind=REAL64), intent(in)                  :: dt
        
        real   (kind=REAL64), dimension(2)                :: x_temp, k1, k2, k3, k4
        real   (kind=REAL64)                              :: halfdt        
        integer(kind= INT32)                              :: i

        halfdt = dt/2
        ! Add openacc directives

        do i = 1, 2
           x_temp(i) = pop(i)%state(1)
        enddo

        call Derivee(x_temp, k1, pop)
        ! Add openacc directives
        do i = 1, 2
             x_temp(i) = pop(i)%state(1) + k1(i)*halfdt
        enddo
        
        call Derivee(x_temp, k2, pop)
        ! Add openacc directives
        do i = 1, 2
            x_temp(i) = pop(i)%state(1) + k2(i)*halfdt
        enddo

        call Derivee(x_temp, k3, pop)
        ! Add openacc directives
        do i = 1, 2
            x_temp(i) = pop(i)%state(1) + k3(i)*dt
        enddo

        call Derivee(x_temp, k4, pop)
        ! Add openacc directives
        do i = 1, 2
            pop(i)%state(1) = pop(i)%state(1) + (dt/6.0)*(k1(i) + 2.0*k2(i) + 2.0*k3(i) + k4(i))
        enddo
    end subroutine rk4

end module utils_lokta

program lokta_volterra
    use utils_lokta
    use openacc
    implicit none

    type   (population ), dimension(2)         :: pred_prey
    real   (kind=REAL64)                       :: ti, tf, dt, tmax
    integer                                    :: i
    
    ti   =   0.00 
    dt   =   0.05 
    tmax = 100.00 
    
    allocate(pred_prey(2)%state(3), pred_prey(1)%state(3)) 
    
    pred_prey(2)%state(1) = 15.00  ! predator count
    pred_prey(2)%state(2) = 0.01   ! predator birth rate
    pred_prey(2)%state(3) = 1.0    ! predator death rate

    pred_prey(1)%state(1) = 100.00 ! prey count
    pred_prey(1)%state(2) = 2.0    ! prey birth rate 
    pred_prey(1)%state(3) = 0.02   ! prey death rate

    do i=1,2
    ! Add openacc directives to pass child structure to GPU first
    enddo
    ! Add openacc directives for parent structure and connection to child

    do while (ti <= tmax)
        tf = ti + dt
        call rk4(pred_prey, dt)
        do i = 1, 2
            ! Add openacc directives
        enddo
        write(*,*) tf, pred_prey(1)%state(1), pred_prey(2)%state(1)
        ti = tf
    end do    
    do i=1,2
    ! Add openacc directives 
    enddo
    ! Add openacc directives
end program lokta_volterra    

## Solution

In [ ]:
%%idrrun -a
!!  examples/Fortran/Deep_copy_attach_detach_solution.f90
module utils_lokta
    use ISO_FORTRAN_ENV, only : REAL64, INT32
    implicit none

    type :: population
        real(kind=REAL64),dimension(:),allocatable :: state
    end type          

    contains
    subroutine derivee(x, dx, pop)
        type   (population ), dimension(2), intent(in)  :: pop 
        real   (kind=REAL64), dimension(2), intent(in)  :: x
        real   (kind=REAL64), dimension(2), intent(out) :: dx
        !$acc data present(x(:), dx(:), pop(:), pop(1)%state(:), pop(2)%state(:))
        !$acc serial
        dx(1) =  pop(1)%state(2)*x(1) - pop(1)%state(3)*x(1)*x(2)
        dx(2) = -pop(2)%state(3)*x(2) + pop(2)%state(2)*x(1)*x(2)
        !$acc end serial
        !$acc end data
    end subroutine derivee

    subroutine rk4(pop, dt)
        type   (population ), dimension(2), intent(inout) :: pop    
        real   (kind=REAL64), intent(in)                  :: dt
        
        real   (kind=REAL64), dimension(2)                :: x_temp, k1, k2, k3, k4
        real   (kind=REAL64)                              :: halfdt        
        integer(kind= INT32)                              :: i

        halfdt = dt/2
        !$acc data create(k1(:), k2(:), k3(:), k4(:), x_temp(:)) present(pop(:), pop(1)%state(:), pop(2)%state(:))

        !$acc parallel loop
        do i = 1, 2
           x_temp(i) = pop(i)%state(1)
        enddo

        call Derivee(x_temp, k1, pop)
        !$acc parallel loop
        do i = 1, 2
             x_temp(i) = pop(i)%state(1) + k1(i)*halfdt
        enddo
        
        call Derivee(x_temp, k2, pop)
        !$acc parallel loop
        do i = 1, 2
            x_temp(i) = pop(i)%state(1) + k2(i)*halfdt
        enddo

        call Derivee(x_temp, k3, pop)
        !$acc parallel loop
        do i = 1, 2
            x_temp(i) = pop(i)%state(1) + k3(i)*dt
        enddo

        call Derivee(x_temp, k4, pop)
        !$acc parallel loop
        do i = 1, 2
            pop(i)%state(1) = pop(i)%state(1) + (dt/6.0)*(k1(i) + 2.0*k2(i) + 2.0*k3(i) + k4(i))
        enddo
       !$acc end data
    end subroutine rk4

end module utils_lokta

program lokta_volterra
    use utils_lokta
    use openacc
    implicit none

    type   (population ), dimension(2)         :: pred_prey
    real   (kind=REAL64)                       :: ti, tf, dt, tmax
    integer                                    :: i
    
    ti   =   0.00 
    dt   =   0.05 
    tmax = 100.00 
    
    allocate(pred_prey(2)%state(3), pred_prey(1)%state(3)) 
    
    pred_prey(2)%state(1) = 15.00  ! predator count
    pred_prey(2)%state(2) = 0.01   ! predator birth rate
    pred_prey(2)%state(3) = 1.0    ! predator death rate

    pred_prey(1)%state(1) = 100.00 ! prey count
    pred_prey(1)%state(2) = 2.0    ! prey birth rate 
    pred_prey(1)%state(3) = 0.02   ! prey death rate

    do i=1,2
       !$acc enter data copyin(pred_prey(i)%state(:))
    enddo
    !$acc enter data copyin(pred_prey(1:2)) attach(pred_prey(1)%state(:), pred_prey(2)%state(:))

    do while (ti <= tmax)
        tf = ti + dt
        call rk4(pred_prey, dt)
        do i = 1, 2
            !$acc update self(pred_prey(i)%state(1))
        enddo
        write(*,*) tf, pred_prey(1)%state(1), pred_prey(2)%state(1)
        ti = tf
    end do    
    do i=1,2
        !$acc exit data detach(pred_prey(i)%state)
        !$acc exit data delete(pred_prey(i)%state)
    enddo
    !$acc exit data delete(pred_prey(:))

end program lokta_volterra    